In [1]:
import QDNS
import logging

In [2]:
class Alice(QDNS.Device):
    def __init__(self):
        super().__init__("Alice")
        self.create_new_application(self.alice_default_app)

    @staticmethod
    def alice_default_app(app: QDNS.Application, *user_args):
        qubits = app.send_entangle_pairs(128, "Bob", routing=True)["my_pairs"]
        results = app.measure_qubits(qubits)["results"]
        app.put_simulation_result(results)

In [3]:
class Bob(QDNS.Device):
    def __init__(self):
        super().__init__("Bob")
        self.create_new_application(self.bob_default_app)

    @staticmethod
    def bob_default_app(app: QDNS.Application, *user_args):
        qubits = app.wait_next_qubits(128)["qubits"]
        results = app.measure_qubits(qubits)["results"]
        app.put_simulation_result(results)

In [4]:
def main():
    logging.basicConfig(level=logging.WARNING)

    alice = Alice()
    bob = Bob()

    net = QDNS.Network(alice, bob)
    net.add_channels(alice, bob, length=10.0) # 10km.

    frames = {
        1: 256,
        2: 256
    }

    QDNS.change_default_cirq_qframe_configuretion(frames)

    sim = QDNS.Simulator()
    res = sim.simulate(net, backend=QDNS.CIRQ_BACKEND)
    
    alice_result = res.user_dumpings("Alice", QDNS.DEFAULT_APPLICATION_NAME)
    bob_result = res.user_dumpings("Bob", QDNS.DEFAULT_APPLICATION_NAME)
    
    match = 0
    for i in range(alice_result.__len__()):
        if alice_result[i] == bob_result[i]:
            match += 1
            
    print("Measurement results match rate: {}%.".format(match/alice_result.__len__()*100))

In [5]:
main()

Measurement results match rate: 84.375%.
